In [2]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [62]:
df=pd.read_excel('filtered_data2.xlsx')
pie_data = pd.DataFrame({'證券代碼': df['證券代碼'], '產業名稱':df['產業名稱'],'區間股價變化率(最高價)':df['區間股價變化率(最高價)'] })
# 定義自定義函數判斷上漲或下跌
def label_trend(row):
    return '上漲' if row['區間股價變化率(最高價)'] >= 0 else '下跌'

# 新增一個欄位 "趨勢"
pie_data['漲跌'] = pie_data.apply(label_trend, axis=1)
pie_data['Counts']=1

# # 計算類別數量
# category_counts = df['產業名稱'].value_counts()
# pie_data = pd.DataFrame({'產業名稱': category_counts.index, 'Counts': category_counts.values})
# # 計算同一產業下，區間股價變化率(最高價)>0和<=0的數量
# positive_counts = []
# negative_counts = []

# for industry in pie_data['產業名稱']:
#     positive_count = len(df[(df['產業名稱'] == industry) & (df['區間股價變化率(最高價)'] > 0)])
#     negative_count = len(df[(df['產業名稱'] == industry) & (df['區間股價變化率(最高價)'] <= 0)])
#     positive_counts.append(positive_count)
#     negative_counts.append(negative_count)

# pie_data['上漲家數'] = positive_counts
# pie_data['下跌家數'] = negative_counts
pie_data

,證券代碼,產業名稱,區間股價變化率(最高價),漲跌,Counts
0,1103,水泥工業,26.60,上漲,1
1,1319,汽車工業,-27.35,下跌,1
2,1436,建材營造,10.62,上漲,1
3,1528,電機機械,69.41,上漲,1
4,2360,其他電子業,51.87,上漲,1
...,...,...,...,...,...
616,6874,資訊服務業,63.43,上漲,1
617,8279,生技醫療,20.12,上漲,1
618,8424,建材營造,4.92,上漲,1
619,8463,其他,26.36,上漲,1


In [58]:

fig = px.sunburst(pie_data, path=['產業名稱', '漲跌'], values='Counts', custom_data=['Counts'])

fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))
fig.update_traces(textinfo='label+value+percent parent')

hover_template = f'<b>%{{label}}</b><br>Counts: %{{customdata[0]}}<br>佔比: %{{percentParent:.0%}}'
fig.update_traces(hovertemplate=hover_template)
fig.show()


In [4]:
col_name_b="最高價當日-200-400 張(比率)變化"
row_name_b="指數名稱"
max_length = 6

col_name_b = col_name_b[:max_length] + ('\n' if len(col_name_b) > max_length else '') + col_name_b[max_length:]
row_name_b = row_name_b[:max_length] + ('\n' if len(row_name_b) > max_length else '') + row_name_b[max_length:]
row_name_b

'指數名稱'

In [9]:
import numpy as np
import os
DATA_DIR = ''
FILE_NAME = 'filtered_data.xlsx'
data_path = os.path.join(DATA_DIR, FILE_NAME)
df = pd.read_excel(data_path)
# max_value= df['區間股價變化率(最高價)'].max()
# min_value = df['區間股價變化率(最高價)'].min()
# 將空值（NaN）替換為0
df['區間股價變化率(最高價)'].fillna(0, inplace=True)
# 將無窮大值（inf）替換為除了inf外的最大值
max_value_without_inf = df['區間股價變化率(最高價)'].replace([np.inf, -np.inf], np.nan).max(skipna=True)
df['區間股價變化率(最高價)'].replace([np.inf, -np.inf], max_value_without_inf, inplace=True)
# 檢查空值
# has_nan = df['區間股價變化率(最高價)'].isna().any()
# has_nan
# 檢查無窮大值
# has_inf = np.isinf(df['區間股價變化率(最高價)']).any()
# has_inf
max_value = np.clip(df['區間股價變化率(最高價)'].max(), 10, 50)
min_value = np.clip(df['區間股價變化率(最高價)'].min(), -10, -50)
df=df[df['區間股價變化率(最高價)'].notnull() | df['區間股價變化率(最高價)'].notna()]
df['資本額(千萬)'] = (df['資本額']/10000000).round(2)

fig = px.treemap(
    df,
    path=['產業名稱','公司'],  # category型
    values='資本額(千萬)', 
    color='區間股價變化率(最高價)', 
    hover_data=['證券代碼','公司', '資本額', '規模', '產業名稱', '區間股價變化率(最高價)', '區間股價變化率(最高價)'],
    range_color=[min_value,max_value],
    # range_color=[-50, 50],
    color_continuous_scale='RdYlGn_r', 
    color_continuous_midpoint=0 , 
)
fig.update_traces(textinfo='label+value',textfont = dict(size = 10)) 
fig.write_html('graph/1/my_plot.html')
fig.show()